In [6]:
%pip install pandas
%pip install sentence_transformers
%pip install spacy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.3 MB/s eta 0:00:00
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 46.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.4/488.4 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [31]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import spacy

In [32]:
%pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [33]:
device_id = 4
print(torch.cuda.is_available())
device = torch.device("cpu")
# torch.cuda.set_device(torch.device(f'cuda:{device_id}' if torch.cuda.is_available() else 'cpu'))
# print ('Cuda device %s | %s | %s/%sGB' % (torch.cuda.current_device(), torch.cuda.get_device_name(device_id),round(torch.cuda.memory_allocated(device_id)/1024**3,1),round(torch.cuda.memory_reserved(device_id)/1024**3,1)))

False


In [34]:
nlp = spacy.load("en_core_web_lg")

def spacy_tokenizer(doc):
    tokens = nlp(doc)
    return([token.lemma_.lower() for token in tokens if (token.text.isalnum() and not token.is_stop and not token.is_punct and not token.like_num)])
    

In [35]:
INPUT_FILE_NAME = 'Newsfeed_raw.json'

In [36]:
article_df = pd.read_json(INPUT_FILE_NAME) 

In [37]:
article_df.dropna(subset=['text','title'],inplace=True)

In [38]:
article_df.columns = ['id', 'date', 'title', 'text', 'story'] # drop story column if not available

In [39]:
article_df = article_df.head(1000)
article_df.head()

,id,date,title,text,story
0,0,2014-01-01,Chinese obstetrician admits to selling patient...,A Chinese doctor has admitted in court that sh...,91630
1,1,2014-01-01,Astronauts ring in the New Year at Times Square,Legions of onlookers watched the big screen in...,46837
2,2,2014-01-01,Trapped ship passengers can't go overboard wit...,But they can't party too hard because the resc...,38176
3,3,2014-01-01,Beyonce shuttle sample angers Nasa,TRAGEDY: The world watched in horror as the Ch...,95841
4,4,2014-01-01,Justice delays health law's birth control mandate,"Only hours before the law was to take effect, ...",96661


In [41]:
article_df['sentences'] = [[t] for t in article_df.title]
article_df['sentence_counts'] = ""
article_df['sentence_tokens'] = [[spacy_tokenizer(t)] for t in article_df.title]

In [43]:
all_sentences = []
all_sentence_tokens = []
for text in article_df['text'].values:
    parsed = nlp(text)
    sentences = []
    sentence_tokens = []
    for s in parsed.sents:
        if len(s) > 1:
            sentences.append(s.text)
            sentence_tokens.append([token.lemma_.lower() for token in s if (token.text.isalnum() and not token.is_stop and not token.is_punct and not token.like_num)])
    all_sentences.append(sentences)
    all_sentence_tokens.append(sentence_tokens)



In [44]:
for i in range(len(all_sentences)):
    article_df.at[i,'sentences'] = article_df.loc[i].sentences + all_sentences[i]
    article_df.at[i,'sentence_tokens'] = article_df.loc[i].sentence_tokens + all_sentence_tokens[i]
    article_df.at[i,'sentence_counts'] = len(article_df.loc[i].sentences)

In [46]:
st_model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')
#https://www.sbert.net/docs/pretrained_models.html

In [50]:
article_df['sentence_tokens'][5]

[['supreme',
  'court',
  'justice',
  'delay',
  'birth',
  'control',
  'mandate',
  'health',
  'care',
  'law',
  'catholic',
  'group'],
 ['washington',
  'hour',
  'law',
  'effect',
  'supreme',
  'court',
  'justice',
  'tuesday',
  'block',
  'implementation',
  'president',
  'barack',
  'obama',
  'health',
  'care',
  'law',
  'force',
  'religion',
  'affiliate',
  'organization',
  'provide',
  'health',
  'insurance',
  'employee',
  'include',
  'birth',
  'control'],
 ['justice',
  'sonia',
  'sotomayor',
  'decision',
  'come',
  'flurry',
  'effort',
  'catholic',
  'affiliate',
  'group',
  'nation'],
 ['group',
  'rush',
  'federal',
  'court',
  'stop',
  'wednesday',
  'start',
  'portion',
  'affordable',
  'care',
  'act',
  'know',
  'obamacare'],
 ['sotomayor',
  'act',
  'request',
  'organization',
  'catholic',
  'nun',
  'denver',
  'little',
  'sisters',
  'poor',
  'home',
  'aged'],
 ['request',
  'emergency',
  'stay',
  'deny',
  'early',
  'day',
  

In [51]:
embeddings = []
errors = []
k = 0
for sentences in article_df['sentences']:
    try:
        embedding = st_model.encode(sentences)
        embeddings.append(embedding)
    except Exception as e:
        errors.append(k)
        print("error at", k, e)

    k = k + 1
    if k % 100 ==0:
        print(k)

100
200
300
400
500
600
700
800
900
1000


In [52]:
article_df['sentence_embds'] = embeddings

In [53]:
# for (idx,row) in article_df.iterrows():
#     for n in noise_list:
#         if n in row['sentences']:
#             article_df.drop(idx, inplace = True)
#             break

NameError: name 'noise_list' is not defined

In [54]:
article_df['date'] = [str(k)[:10] for k in article_df['date']]

In [55]:
article_df.sort_values(by=['date'],inplace=True)

In [56]:
article_df.reset_index(inplace= True, drop=True)

In [57]:
article_df['id'] = article_df.index

In [58]:
OUTPUT_FILE_NAME = 'Newsfeed_raw_output.json'

In [59]:
article_df.to_json(OUTPUT_FILE_NAME)